In [ ]:
# importando tudo que a gente precisa importar
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import IPython.display as ipd

In [ ]:
# ajustando nossos plots
plt.rcParams['figure.figsize'] = (15,5)

# Representações

## representações no domínio do tempo

a forma de onda é a representação mais simples que temos. é basicamente o plot de todas as amplitudes do nosso vetor de floats. ela pode trazer algumas informações interessantes, mas no geral só é útil quando temos sinais bem simples e, preferencialmente, de um único instrumento.

isso acontece porque sinais de fontes diferentes (por exemplo, um baixo e uma bateria) acabam se somando no sinal final e, ao visualizarmos a forma de onda, não conseguiremos distinguir um do outro.

In [ ]:
def cria_sinal(duracao, frequencia_amostragem, frequencia):
    t = np.arange(0,duracao,1/frequencia_amostragem)
    x = np.e**(-1j*2*np.pi*frequencia*t)
    
    return t, x

In [ ]:
duracao = 3 # segundos
taxa_amostragem = 22050 # amostras por segundo
frequencia = 440 # Hz

a4 = librosa.tone(frequencia, sr=taxa_amostragem, duration=duracao)

In [ ]:
ipd.Audio(a4, rate=22050)

In [ ]:
flute_a4, flute_fs = librosa.load('./audios/flute-a4.wav') 
ipd.Audio(flute_a4, rate=flute_fs)

In [ ]:
oboe_a4, oboe_fs = librosa.load('./audios/oboe-a4.wav')
ipd.Audio(oboe_a4, rate=oboe_fs)

In [ ]:
plt.subplot(2,1,1)
plt.plot(a4[0:200])
plt.title('A4 sintetizado')
plt.subplot(2,1,2)
plt.plot(flute_a4[0:200])
plt.title('A4 flauta')

plt.tight_layout()

### Um exemplo um pouco mais real

In [ ]:
# listando alguns exemplos de áudio disponíveis na biblioteca
librosa.util.list_examples()

In [ ]:
trumpet = librosa.util.example('trumpet')

In [ ]:
x, fs = librosa.core.load(trumpet)

In [ ]:
x.shape, fs, len(x)/fs

In [ ]:
ipd.Audio(x, rate=fs)

In [ ]:
librosa.display.waveplot(x)

In [ ]:
ipd.Audio(x, rate=fs)

In [ ]:
# vamos pegar um trecho menor pra ver mais de perto
init_sample = 2*fs
end_sample = init_sample + fs
x_frame = x[init_sample:end_sample]

plt.plot(x_frame)

In [ ]:
# hmMmmMmMMMM, mais zoom
init_sample = 2*fs + fs//16
end_sample = init_sample + fs//32
x_frame = x[init_sample:end_sample]

print(x_frame.shape)
librosa.display.waveplot(x_frame)

In [ ]:
plt.plot(x_frame)

In [ ]:
# o último zoom, eu juro
plt.plot(x_frame[50:250])

In [ ]:
ipd.Audio(x_frame, rate=fs)

## representação no domínio da frequência 

a transformada de Fourier parte do princípio de que um sinal pode ser representado como uma soma de outros sinais e que a gente consegue encontrar quais são as componentes que fazem parte dessa soma. 

a transformada discreta de Fourier (DFT) é uma ferramenta que nos permite visualizar o nosso sinal do domínio da frequência. é bastante útil pra conseguirmos entender quais as frequências mais importantes da janela de tempo que estamos analisando.

a transformada rápida de Fourier (Fast Fourier Transform, carinhosamente conhecida como FFT) é uma versão otimizada do algoritmo e é o que geralmente usamos pra fazer a visualização.

o resultado dessa operação é chamando de "espectro" e ele perde toda a informação temporal que tínhamos antes.

de uma forma bem simples, a intuição da transformada de Fourier é comparar o nosso sinal com diversas senoides de frequências diferentes e ir "anotando" o quão parecida aquela onda é do nosso sinal. quanto maior o coeficiente, mais parecida a onda de frequência `k` é do nosso sinal. 

podemos ter diversas ondas parecidas com o sinal quando, por exemplo, tocamos um acorde de violão. 

In [ ]:
# FFT da nota A4 sintetizada
A4 = np.fft.fft(a4)
FA4 = np.fft.fft(flute_a4)

plt.subplot(2,1,1)
N_a = len(a4)
plt.plot(np.arange(N_a/2)*taxa_amostragem/N_a,np.abs(A4[0:(len(a4)//2)]))
plt.title('A4 sintetizado')
plt.subplot(2,1,2)
plt.title('A4 flauta')
N_f = len(flute_a4)
plt.plot(np.arange(N_f/2)*flute_fs/N_f, np.abs(FA4[0:(len(flute_a4)//2)]))

plt.tight_layout()

In [ ]:
# FFT de um trechinho do áudio do trompete
X_frame = np.abs(np.fft.fft(x_frame))
N = len(x_frame)
x_axis_hz = np.arange(N/2)*fs/N    # transformação pra representar o eixo x em Hertz
plt.plot(x_axis_hz, X_frame[0:X_frame.shape[0]//2+1])
plt.title('Espectro de magnitude')

### o que mais conseguimos fazer diretamente no domínio da frequência?

uma tarefa que às vezes compensa é remover algumas das componentes de frequências, geralmente as componentes mais fracas, que potencialmente podem ser "lixo".

In [ ]:
# remoção das componentes mais fracas
X = np.fft.fft(x)
Y = X.copy();
N = len(X)
threshold = 0.1*abs(Y).max()

for val in np.nditer(Y, op_flags=['readwrite']):
    if abs(val) < threshold:
        val[...] = 0
        
reconstrucao = np.real(np.fft.ifft(Y)); 

plt.subplot(2,2,1)
plt.plot(x)
plt.title('Audio Original')

plt.subplot(2,2,2)
plt.plot(reconstrucao)
plt.title('Audio Ressintetizado')


plt.subplot(2,2,3)
plt.plot(np.abs(X[0:N//2]))
plt.title('Espectro Original')

plt.subplot(2,2,4)
plt.plot(np.abs(np.fft.fft(reconstrucao)[0:N//2]))
plt.title('Espectro Ressintetizado')

plt.tight_layout()

ipd.Audio(reconstrucao, rate=fs)

## representações tempo-frequência

uma forma esperta de conseguir ver as informações de frequência em função do tempo é janelar o sinal, ou seja, analisar pequenos trechos por vez e ir concatenando as informações que achamos. o nome dessa representação é "espectrograma"

o espectrograma é uma representação muito usada de sinais de áudio. alguns sistemas de aprendizado de máquina inclusive aplicam métodos de visão computacional e processamento de imagens pra conseguir informações sobre o áudio, uma vez que a imagem é mais leve que o áudio e o processamento é mais barato.

In [ ]:
plt.figure()
img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(librosa.stft(x)), ref=np.max), y_axis='log', x_axis='time')
plt.title('espectrograma do audio do trompete')
plt.colorbar(format="%+2.f dB")

In [ ]:
plt.figure()
img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(librosa.stft(flute_a4)), ref=np.max), y_axis='log', x_axis='time')
plt.title('espectrograma do audio da flauta')
plt.colorbar(format="%+2.f dB")

In [ ]:
plt.figure()
img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(librosa.stft(a4)), ref=np.max), y_axis='log', x_axis='time')
plt.title('espectrograma do audio sintetizado')
plt.colorbar(format="%+2.f dB")

## Separação harmônica e percussiva

In [ ]:
def plota_espectrogramas(X, X_harmonico, X_percussivo):
    fig, ax = plt.subplots(nrows=3, sharex=True, sharey=True)

    img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(X), ref=reference_power),
                             y_axis='log', x_axis='time', ax=ax[0])
    ax[0].set(title='Espectrograma')
    ax[0].label_outer()

    librosa.display.specshow(librosa.amplitude_to_db(np.abs(X_harmonico), ref=reference_power),
                             y_axis='log', x_axis='time', ax=ax[1])
    ax[1].set(title='Espectrograma harmônico')
    ax[1].label_outer()

    librosa.display.specshow(librosa.amplitude_to_db(np.abs(X_percussivo), ref=reference_power),
                             y_axis='log', x_axis='time', ax=ax[2])
    ax[2].set(title='Espectrograma percussivo')
    fig.colorbar(img, ax=ax)

In [ ]:
praieira, fs = librosa.load('audios/chico_science-praieira.mp3')

In [ ]:
ipd.Audio(praieira, rate=fs)

In [ ]:
trecho_praieira = praieira[5*fs:15*fs]

In [ ]:
ipd.Audio(trecho_praieira, rate=fs)

In [ ]:
X = librosa.stft(trecho_praieira)

In [ ]:
harmonico, percussivo = librosa.decompose.hpss(X)

In [ ]:
reference_power = np.max(np.abs(X))

In [ ]:
plota_espectrogramas(X, harmonico, percussivo)

In [ ]:
harmonico_ress = librosa.istft(harmonico.real)

In [ ]:
ipd.Audio(harmonico_ress, rate=fs)

In [ ]:
percussivo_ress = librosa.istft(percussivo.real)
ipd.Audio(percussivo_ress, rate=fs)

In [ ]:
choice_path = librosa.util.example('choice')
choice, c_fs = librosa.load(choice_path)

In [ ]:
ipd.Audio(choice, rate=c_fs)

In [ ]:
C = librosa.stft(choice)
C_harm, C_perc = librosa.decompose.hpss(C)

reference_power = np.max(np.abs(C))

plota_espectrogramas(C, C_harm, C_perc)

In [ ]:
c_harm_ress = librosa.istft(C_harm)
c_perc_ress = librosa.istft(C_perc)

In [ ]:
ipd.Audio(c_harm_ress, rate=c_fs)

In [ ]:
ipd.Audio(c_perc_ress, rate=c_fs)

In [ ]:
# this is reconstrução perfeita, bitches
ipd.Audio(c_harm_ress + c_perc_ress, rate=c_fs)

In [ ]:
# ultima brincadeira
ungrateful, u_fs = librosa.load('audios/escape_the_fate-ungrateful.mp3')

In [ ]:
ipd.Audio(ungrateful, rate=u_fs)

In [ ]:
U = librosa.stft(ungrateful)
U_harm, U_perc = librosa.decompose.hpss(U)

plota_espectrogramas(U, U_harm, U_perc)

In [ ]:
u_harm_ress = librosa.istft(U_harm)
u_perc_ress = librosa.istft(U_perc)

In [ ]:
ipd.Audio(u_harm_ress, rate=u_fs)

In [ ]:
ipd.Audio(u_perc_ress, rate=u_fs)